In [62]:
# COVID-19 Simulations

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
import geopandas as gpd
from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel, Select, Button, Legend
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider,PrintfTickFormatter)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_20

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()
import random
import pandas as pd
import numpy as np
import colorcet as cc
import random
import pandas as pd
import numpy as np

num_days = 100
regions = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17']
attributes = ['infected', 'hospitalized', 'dead', 'quarantined','days', 'cumulative']


palette = [cc.rainbow[i*15] for i in range(17)]

df_kz = gpd.read_file('KAZ_adm1.shp')
geosource = GeoJSONDataSource(geojson = df_kz.to_json())




# code for testing
def create_the_data_for_each_region():
    df = pd.DataFrame()
    for i, region in enumerate(regions):
        rand_infected = np.random.randint(low=1, high=100000, size=20)
        rand_infected = np.sort(rand_infected)
        days = np.linspace(1,20,20)
        rand_population = max(rand_infected)*2
        rand_hospitalized = np.ceil(rand_infected/5)
        rand_dead = np.ceil(rand_infected/50)
        rand_quarantined = np.ceil(rand_infected*1.2)
        cumulative = np.cumsum(rand_infected)
        dict = {'cumulative': cumulative,'days': days, 'region': region, 'infected': rand_infected, 'population': rand_population, 
                'hospitalized': rand_hospitalized, 'dead': rand_dead, 'quarantined':rand_quarantined}
        df = df.append(pd.DataFrame(dict)) 
    return df

df = create_the_data_for_each_region()
available_regions = list(df['region'].unique())
available_attributes = attributes

def modify_doc(doc):
    def remote_jupyter_proxy_url(port):

        base_url = os.environ['EXTERNAL_URL']
        host = urllib.parse.urlparse(base_url).netloc

        # If port is None we're asking for the URL origin
        # so return the public hostname.
        if port is None:
            return host

        service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
        proxy_url_path = 'proxy/%d' % port

        user_url = urllib.parse.urljoin(base_url, service_url_path)
        full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    
    

    def make_dataset_1(region):
        subset = df[df['region']==region]
        return ColumnDataSource(subset)
    def style(p):
            # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'
        
            # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

            # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p


    def make_plot(src):
        
        p = figure(plot_width=600, plot_height=600, title='COVID-19 Simulation',
                  x_axis_label = 'Days', y_axis_label = 'Number of people',toolbar_location="above")
        p1 = figure(title = ' Kazakhstan', plot_height=600, plot_width=650, background_fill_color='black',background_fill_alpha = 0.8, toolbar_location='above')
        p1.xgrid.grid_line_color=None
        p1.ygrid.grid_line_color=None
        states=p1.patches('xs','ys', source=geosource, fill_color='red', line_color='gray')
         #  p.vbar_stack(regions,source=src,x='days',  
            # color='black', width=0.75, 
            # legend='Daily Infected')

        r0 = p.line(source=src,x='days', y='hospitalized', 
              color='white', line_width=1, line_dash='dashed',
              legend='Hospitalized')
        r1 = p.circle(source=src,x='days', y='hospitalized', color='white', size= 10, legend='Hospitalized')
        
        r2 = p.line(source=src,x='days', y='infected', 
              color='red', line_width=1, line_dash ='dotted',
              legend='Infected')
        r3 = p.circle(source=src,x='days', y='infected', 
              color='red', size=10,
              legend='Infected')
        
        r4 = p.line(source=src,x='days', y='quarantined', 
              color='blue', line_width=1, line_dash='dashdot',
              legend='Quarantined')
        r5 = p.circle(source=src,x='days', y='quarantined', 
              color='blue', size=10,
              legend='Quarantined')
        
        r6 = p.line(source=src,x='days', y='dead', 
              color='green', line_width=1, line_dash='dotted',
              legend='Dead')
        r7 = p.circle(source=src,x='days', y='dead', 
              color='green', size=10,
              legend='Dead')

        p.background_fill_color = "black"
        p.background_fill_alpha = 0.8
        p.legend.location = "top_left"
        p.legend.background_fill_color = "cyan"
        p.legend.background_fill_alpha = 0.9
        p.outline_line_width = 7
        p.outline_line_alpha = 0.9
        p.outline_line_color = "black"
        p1.outline_line_width = 7
        p1.outline_line_alpha = 0.9
        p1.outline_line_color = "black"
        p.xaxis[0].formatter = PrintfTickFormatter(format="%9.0f")
        p.yaxis[0].formatter = PrintfTickFormatter(format="%9.0f")
        p.legend.label_text_font_size = '5pt'
        #p.xgrid.grid_line_color = None

        # change just some things about the y-grid
        #p.ygrid.grid_line_alpha = 0.5
        #p.ygrid.grid_line_dash = [0, 0]
        
        p = style(p)
        return p, p1


    #update function for region select

    def update(attr, old, new):
        region = region_selection.value
        #plot.title.text = 'Simulation for' + regions[region]['title']
        
        new_src  = make_dataset_1(region)
        src.data.update(new_src.data)
    
    #Select the region for plotting
    initial_region = '01'
    region_selection = Select(value=initial_region, title='Region Selection', options=regions)
    region_selection.on_change('value', update)


   # region_selection = CheckboxGroup(labels=available_regions, active= [0,1])
   # region_selection.on_change('active', update)
    #Select attributes to plot

    #Select initial for simulation
    test_slider = Slider(start=1,end=10,step=1,value=5, title='Mortality Rate')
    test_slider.on_change('value', update)
    
    test_slider_1 = Slider(start=1,end=10,step=1,value=5, title='Infection Rate')
    test_slider_1.on_change('value', update)
    
    test_slider_2 = Slider(start=1,end=10,step=1,value=5, title='Hospitalization Rate')
    test_slider_2.on_change('value', update)
    
    bt = Button(label='Simulate with new initial conditions')

    def change_click():
        print('I was clicked')

    bt.on_click(change_click)


    
    src = make_dataset_1(initial_region)
    
    p, p1 = make_plot(src)
    
    controls = WidgetBox(region_selection, test_slider_1, test_slider_2, test_slider, bt)
    #layout = column(controls, p,p1)
    #layout = column(layout, p1)
    layout = row(p,p1)
    layout = column(controls, layout)
    tab = Panel(child=layout, title = 'Covid Simulation')
    tabs = Tabs(tabs=[tab])
    doc.add_root(tabs)
    
    
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app,notebook_url='localhost:8888')

Loading BokehJS ...

ERROR:fiona._env:Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.
